# Model Translation + APS Viewer with Tree Overlays

This notebook demonstrates how to upload a Revit model with `aps_automation_sdk`, translate it to SVF/SVF2, and add custom 3D tree meshes to the scene using the **`OverlayMeshes` plugin** from `aps_viewer_sdk`.

## What you'll learn:
- Upload and translate a Revit model using `aps_automation_sdk` helpers
- Use the **`OverlayMeshes` plugin** (from `aps_viewer_sdk.plugins`) to add custom 3D geometries
- Create stylized trees using boxes (trunks) and cones (canopy layers)
- Overlay custom meshes on top of the APS Viewer scene

**Prereqs**
- Env vars: `CLIENT_ID`, `CLIENT_SECRET`
- Dependencies: `aps_viewer_sdk`, `aps_automation_sdk`, `requests`
- A local Revit file to upload (SampleModel.rvt)

In [1]:
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

# Import from aps_viewer_sdk
from aps_viewer_sdk.helper import get_2lo_token
from aps_viewer_sdk import APSViewer
from aps_viewer_sdk.plugins import OverlayMeshes  # Plugin for adding custom 3D meshes

# Import from aps_automation_sdk for uploading and translating models
from aps_automation_sdk import ActivityInputParameter, translate_file_in_oss

load_dotenv()

True

In [2]:
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError("CLIENT_ID and CLIENT_SECRET must be set in environment")

## Upload and translate the model

Upload the Revit model to APS and translate it for viewing.

In [4]:
from aps_automation_sdk import ActivityInputParameter

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
if not client_id or not client_secret:
    raise RuntimeError(
        "CLIENT_ID and CLIENT_SECRET must be set in the environment variables."
    )

# Resolve the sample file path
sample_path = Path("SampleModel.rvt").resolve()

print("Starting model processing workflow...")
print(f"Uploading file: {sample_path.name}")

token = get_2lo_token(client_id, client_secret)

bucket_key = uuid.uuid4().hex
object_key = f"input_{uuid.uuid4()}{sample_path.suffix}"

input_param = ActivityInputParameter(
    name="inputFile",
    localName=sample_path.name,
    verb="get",
    description="Input CAD model",
    required=True,
    is_engine_input=False,
    bucketKey=bucket_key,
    objectKey=object_key,
)

input_param.upload_file_to_oss(file_path=str(sample_path), token=token)

# Use the helper from aps_automation_sdk to translate the file
viewer_urn = translate_file_in_oss(
    token=token,
    bucket_key=bucket_key,
    output_object_key=object_key,
    max_wait_time=300,
    poll_interval=15,
    verbose=True,
)
print(f"Viewer URN: {viewer_urn}")

Starting model processing workflow...
Uploading file: SampleModel.rvt
🎯 STARTING MODEL TRANSLATION FOR VIEWING
Bucket: 86627e2ddd4f4844a1fde365146cb36f
Object: input_385d3fa9-dcab-49cc-a4e4-47635bf43562.rvt
🔗 Object URN: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6ODY2MjdlMmRkZDRmNDg0NGExZmRlMzY1MTQ2Y2IzNmYvaW5wdXRfMzg1ZDNmYTktZGNhYi00OWNjLWE0ZTQtNDc2MzViZjQzNTYyLnJ2dA
🔄 Starting Model Derivative translation job...
📋 Job Details: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6ODY2MjdlMmRkZDRmNDg0NGExZmRlMzY1MTQ2Y2IzNmYvaW5wdXRfMzg1ZDNmYTktZGNhYi00OWNjLWE0ZTQtNDc2MzViZjQzNTYyLnJ2dA
⏳ Monitoring translation progress...
  > MD Status: pending (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (27% complete)
  > MD Status: inprogress (65% complete)
  > MD Status: inprogress (77% complete)
  > MD Status: inprogress (99% complete)
  > MD Status: inprogress (99% complete)
  > MD Status: success (complete

## Create tree overlay meshes using the OverlayMeshes plugin

The **`OverlayMeshes`** plugin (from `aps_viewer_sdk.plugins`) allows you to add custom 3D geometries to your APS Viewer scene. 

In this example, we'll create three stylized trees:
- **Trunks**: Created with `add_box()` method (rectangular boxes)
- **Canopy**: Created with `add_cone()` method (layered cones for a stylized tree effect)

Each tree is positioned at different coordinates and has unique colors for trunk and leaves.

In [5]:
def make_tree_plugin() -> OverlayMeshes:
    """Create an OverlayMeshes plugin with three stylized trees."""
    trees = OverlayMeshes(scene_id="trees-overlay")

    # Define specifications for three trees with different sizes and positions
    tree_specs = [
        {
            "base": (0, 0, -25),
            "trunk_size": (2.0, 10.0, 2.0),
            "canopy": [
                {"radius": 6.0, "height": 8.0, "z": -25 + 10.0 + 4.0},
                {"radius": 5.0, "height": 6.5, "z": -25 + 10.0 + 8.0},
                {"radius": 4.0, "height": 5.0, "z": -25 + 10.0 + 11.0},
            ],
            "trunk_color": "#8b5a2b",
            "leaves_color": "#2e8b57",
        },
        {
            "base": (18, -12, -22),
            "trunk_size": (1.6, 9.0, 1.6),
            "canopy": [
                {"radius": 5.5, "height": 7.5, "z": -22 + 9.0 + 3.5},
                {"radius": 4.4, "height": 6.0, "z": -22 + 9.0 + 7.0},
                {"radius": 3.5, "height": 4.5, "z": -22 + 9.0 + 9.5},
            ],
            "trunk_color": "#7a4a2a",
            "leaves_color": "#2f7d50",
        },
        {
            "base": (-15, 8, -20),
            "trunk_size": (2.2, 11.0, 2.2),
            "canopy": [
                {"radius": 6.8, "height": 8.5, "z": -20 + 11.0 + 4.2},
                {"radius": 5.5, "height": 7.0, "z": -20 + 11.0 + 8.0},
                {"radius": 4.2, "height": 5.5, "z": -20 + 11.0 + 11.0},
            ],
            "trunk_color": "#654321",
            "leaves_color": "#3c9b63",
        },
    ]

    # Build each tree
    for spec in tree_specs:
        x, y, z = spec["base"]
        trunk_w, trunk_h, trunk_d = spec["trunk_size"]

        # Add trunk (box)
        trees.add_box(
            (x, y, z + trunk_h / 2.0),
            size=(trunk_w, trunk_h, trunk_d),
            color=spec["trunk_color"],
            opacity=1.0,
            align_to_world_up=True,
        )

        # Add canopy layers (cones)
        for canopy in spec["canopy"]:
            trees.add_cone(
                (x, y, canopy["z"]),
                radius=canopy["radius"],
                height=canopy["height"],
                color=spec["leaves_color"],
                opacity=1.0,
                radial_segments=14,
                align_to_world_up=True,
            )

    return trees


# Create the tree plugin
trees = make_tree_plugin()
print("Created tree overlay plugin with 3 trees")

Created tree overlay plugin with 3 trees


## View the model with tree overlays

Now we create the `APSViewer` instance and add the **`OverlayMeshes` plugin** to it.

The plugin is added using `viewer.add_plugin(trees.spec())`, which injects the custom 3D meshes into the viewer scene. The trees will render as interactive 3D objects overlaid on your Revit model.

In [6]:
# Create the viewer with the translated model
viewer = APSViewer(
    urn=f"urn:adsk.objects:os.object:{bucket_key}/{object_key}",
    token=token,
    views_selector=True,
)

# Add the tree overlay plugin
viewer.add_plugin(trees.spec())

print("Viewer created with tree overlay plugin")
print("The trees will appear as custom 3D meshes in the scene")

# Display the viewer
viewer.show()

Viewer created with tree overlay plugin
The trees will appear as custom 3D meshes in the scene


## Result

Here's what the viewer looks like with custom tree meshes overlaid on the model:

![Tree Overlay Result](../../assets/example2.png)

The `OverlayMeshes` plugin successfully added three stylized trees to the 3D scene. This demonstrates how you can:
- Add custom 3D geometries (boxes, cones, spheres, cylinders) to any model
- Position meshes at specific coordinates in the model space
- Create complex objects by combining primitive shapes
- Overlay contextual elements like trees, furniture, or equipment on architectural models